In [1]:
# !pip install folktables

In [1]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle

## Load pickel file directly

In [2]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
# acs_data = data_source.get_data(states=["AL", "AK", "AZ", "AR", "CA"], download=True)
# acs_data = data_source.get_data(download=True)

In [3]:
state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
               "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
               "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
               "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
               "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# state_codes = ["AL","FL","GA","CA","IL","OH","PA","TX", "NY", ]
# state_codes = [ "CA","FL","GA","IL","OH","PA","TX","NY"]

dfs = {}
for state_code in state_codes:
    acs_data = data_source.get_data(states=[state_code],download=True)
    features, label, group = ACSIncome.df_to_pandas(acs_data)
    dfs[state_code] = (features, label)

In [4]:
all_len=[]
for state_code, (features, label) in dfs.items():
    all_len.append(len(label))
    print(f"State: {state_code}, Features Length: {len(features)}, Label Length: {len(label)}")

State: AL, Features Length: 22268, Label Length: 22268
State: AK, Features Length: 3546, Label Length: 3546
State: AZ, Features Length: 33277, Label Length: 33277
State: AR, Features Length: 13929, Label Length: 13929
State: CA, Features Length: 195665, Label Length: 195665
State: CO, Features Length: 31306, Label Length: 31306
State: CT, Features Length: 19785, Label Length: 19785
State: DE, Features Length: 4713, Label Length: 4713
State: FL, Features Length: 98925, Label Length: 98925
State: GA, Features Length: 50915, Label Length: 50915
State: HI, Features Length: 7731, Label Length: 7731
State: ID, Features Length: 8265, Label Length: 8265
State: IL, Features Length: 67016, Label Length: 67016
State: IN, Features Length: 35022, Label Length: 35022
State: IA, Features Length: 17745, Label Length: 17745
State: KS, Features Length: 15807, Label Length: 15807
State: KY, Features Length: 22006, Label Length: 22006
State: LA, Features Length: 20667, Label Length: 20667
State: ME, Featu

In [5]:
check_df=dfs["AL"][0]
check_df.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P
0,18.0,1.0,18.0,5.0,4720.0,13.0,17.0,21.0,2.0,2.0
1,53.0,5.0,17.0,5.0,3605.0,18.0,16.0,40.0,1.0,1.0
2,41.0,1.0,16.0,5.0,7330.0,1.0,17.0,40.0,1.0,1.0
3,18.0,6.0,18.0,5.0,2722.0,1.0,17.0,2.0,2.0,1.0
4,21.0,5.0,19.0,5.0,3870.0,12.0,17.0,50.0,1.0,1.0


In [6]:
pd.unique(dfs["CA"][0]["RAC1P"][dfs["CA"][0]["RAC1P"].isin([1, 2])])

array([1., 2.])

In [7]:
dfs["CA"][0]

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P
0,30.0,6.0,14.0,1.0,9610.0,6.0,16.0,40.0,1.0,8.0
1,21.0,4.0,16.0,5.0,1970.0,6.0,17.0,20.0,1.0,1.0
2,65.0,2.0,22.0,5.0,2040.0,6.0,17.0,8.0,1.0,1.0
3,33.0,1.0,14.0,3.0,9610.0,36.0,16.0,40.0,1.0,1.0
4,18.0,2.0,19.0,5.0,1021.0,6.0,17.0,18.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...
195660,38.0,1.0,22.0,1.0,1021.0,210.0,0.0,40.0,1.0,6.0
195661,39.0,1.0,22.0,1.0,1021.0,210.0,1.0,40.0,2.0,6.0
195662,61.0,1.0,19.0,1.0,5240.0,17.0,0.0,45.0,1.0,1.0
195663,69.0,7.0,24.0,1.0,2040.0,207.0,0.0,45.0,1.0,6.0


In [8]:
merge_dfs={}
sample_size=2100

for state_code, (features, label) in dfs.items():   

    merge_df = pd.concat([features, label], axis=1)
    merge_df = merge_df.dropna()
    # print(len(merge_df))
    merge_df['PINCP'] = merge_df['PINCP'].replace({True: '>50K', False: '<=50K'})
    merge_df= merge_df[merge_df["RAC1P"].isin([1, 2])]
    # print("Before:: ",pd.unique(merge_df["RAC1P"][merge_df["RAC1P"].isin([1, 2])]))
    
    # Random sampling with maintaning the statistics 
    merge_df['strat'] = merge_df['SEX'].astype(str) + '_' + merge_df['RELP'].astype(str)
    
    # merge_df['strat'] = merge_df['SEX'].astype(str) + '_' + merge_df['RELP'].astype(str) + '_' + merge_df['PINCP'].astype(str)
    # merge_df['strat'] = (
    #     merge_df['COW'].astype(str) + '_' +
    #     merge_df['MAR'].astype(str) + '_' +
    #     merge_df['SEX'].astype(str) + '_' +
    #     merge_df['RAC1P'].astype(str) + '_' +
    #     merge_df['PINCP'].astype(str)
    #     )

    # Check for rare combinations (occur less than 2 times)
             
    combination_counts = merge_df['strat'].value_counts()
    rare_combinations = combination_counts[combination_counts < 2].index
    # print(len(merge_df))
    # print("rare_combinations:",len(rare_combinations))

    # Remove rare combination:- E.g 2.0_14.0_False from FL
    
    merge_df = merge_df[~merge_df['strat'].isin(rare_combinations)]
    # print(len(merge_df))  

    # To handle the splitiing properly
    ratio_of_df= sample_size/len(merge_df)
    if ratio_of_df >= 1.0:
        ratio_of_df = 0.9

    sampled_df, _ = train_test_split(
                        merge_df,
                        train_size=ratio_of_df,
                        stratify=merge_df['strat'],
                        random_state=42 )

    sampled_df = sampled_df.drop('strat', axis=1)
    sampled_df = sampled_df.reset_index(drop=True)

    merge_dfs[state_code] = sampled_df.copy()
    
    # merge_dfs[state_code] = merge_df
    

In [9]:
for state_code, df in merge_dfs.items():
    label_counts = df['PINCP'].value_counts()
    print(f"State: {state_code}, df Length: {len(df)}, Label Counts: {label_counts.to_dict()}")

    # print(f"State: {state_code}, df Length: {len(df)}")

State: AL, df Length: 2100, Label Counts: {'<=50K': 1435, '>50K': 665}
State: AK, df Length: 2100, Label Counts: {'<=50K': 1082, '>50K': 1018}
State: AZ, df Length: 2100, Label Counts: {'<=50K': 1356, '>50K': 744}
State: AR, df Length: 2100, Label Counts: {'<=50K': 1534, '>50K': 566}
State: CA, df Length: 2100, Label Counts: {'<=50K': 1153, '>50K': 947}
State: CO, df Length: 2100, Label Counts: {'<=50K': 1180, '>50K': 920}
State: CT, df Length: 2100, Label Counts: {'<=50K': 1084, '>50K': 1016}
State: DE, df Length: 2100, Label Counts: {'<=50K': 1266, '>50K': 834}
State: FL, df Length: 2100, Label Counts: {'<=50K': 1391, '>50K': 709}
State: GA, df Length: 2100, Label Counts: {'<=50K': 1326, '>50K': 774}
State: HI, df Length: 2100, Label Counts: {'<=50K': 1153, '>50K': 947}
State: ID, df Length: 2100, Label Counts: {'<=50K': 1492, '>50K': 608}
State: IL, df Length: 2100, Label Counts: {'<=50K': 1210, '>50K': 890}
State: IN, df Length: 2100, Label Counts: {'<=50K': 1455, '>50K': 645}
Stat

In [10]:
merge_dfs["CA"].head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,36.0,1.0,16.0,1.0,5330.0,6.0,1.0,40.0,2.0,1.0,>50K
1,41.0,1.0,19.0,1.0,6200.0,303.0,1.0,40.0,1.0,1.0,>50K
2,71.0,1.0,19.0,1.0,600.0,25.0,0.0,12.0,1.0,1.0,>50K
3,40.0,2.0,21.0,1.0,440.0,41.0,0.0,40.0,2.0,1.0,>50K
4,50.0,1.0,12.0,1.0,8320.0,303.0,6.0,40.0,2.0,1.0,<=50K


In [11]:
pd.unique(merge_dfs["AL"]["RAC1P"])

array([1., 2.])

In [15]:
# # Save the dictionary to a file
# with open('dfs.pickle', 'wb') as f:
#     pickle.dump(merge_dfs, f)

In [2]:
with open('dfs.pickle', 'rb') as f:
    dfs_loaded = pickle.load(f)

In [3]:
for state_code, df in dfs_loaded.items():
    # print(df.head())
    print(f"State: {state_code}, df Length: {len(df)}")

State: AL, df Length: 2100
State: AK, df Length: 2100
State: AZ, df Length: 2100
State: AR, df Length: 2100
State: CA, df Length: 2100
State: CO, df Length: 2100
State: CT, df Length: 2100
State: DE, df Length: 2100
State: FL, df Length: 2100
State: GA, df Length: 2100
State: HI, df Length: 2100
State: ID, df Length: 2100
State: IL, df Length: 2100
State: IN, df Length: 2100
State: IA, df Length: 2100
State: KS, df Length: 2100
State: KY, df Length: 2100
State: LA, df Length: 2100
State: ME, df Length: 2100
State: MD, df Length: 2100
State: MA, df Length: 2100
State: MI, df Length: 2100
State: MN, df Length: 2100
State: MS, df Length: 2100
State: MO, df Length: 2100
State: MT, df Length: 2100
State: NE, df Length: 2100
State: NV, df Length: 2100
State: NH, df Length: 2100
State: NJ, df Length: 2100
State: NM, df Length: 2100
State: NY, df Length: 2100
State: NC, df Length: 2100
State: ND, df Length: 2100
State: OH, df Length: 2100
State: OK, df Length: 2100
State: OR, df Length: 2100
S

In [4]:
2100*0.0475

99.75

In [5]:
# NY is removed... We use it for Testing 

state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
               "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
               "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
               "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
               "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [25]:
state_sex_counts = []
state_sex_race_counts=[]

for state_code, df in dfs_loaded.items():
    df_temp=dfs_loaded[state_code].reset_index(drop=True)
    train_data, test_data = train_test_split(df_temp, test_size=0.0475, shuffle=False)

    sex_counts = train_data["SEX"].value_counts()
    
    # Extract male and female counts (fill missing with 0 if not present)
    male_count = sex_counts.get(1.0, 0)
    female_count = sex_counts.get(2.0, 0)
    
    race_counts = train_data["RAC1P"].value_counts()
    white_count = race_counts.get(1.0, 0)
    black_count = race_counts.get(2.0, 0)
    
    # Append the data to the list
    state_sex_race_counts.append({
        "State": state_code,
        "Male_Count": male_count,
        "Female_Count": female_count,
        "White_Count": white_count,
        "Black_Count": black_count
    })
    
    train_data.to_csv(f'50_clients_data/raw_data/{state_code}.data', header=False, index=False)
    
    test_data['PINCP'] = test_data['PINCP'].astype(str) + '.'
    print(test_data.head())

    test_data.to_csv(f'50_clients_data/raw_data/{state_code}.test', header=False, index=False)
    
    # df_temp.to_csv(f'50_clients_data/raw_data/{state_code}.csv', header=False, index=False)

      AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P   PINCP
2000  33.0  1.0  16.0  5.0  9645.0  53.0   0.0  40.0  1.0    1.0  <=50K.
2001  41.0  4.0  19.0  1.0  3870.0   1.0   0.0  40.0  1.0    1.0  <=50K.
2002  35.0  7.0  16.0  1.0  4920.0   1.0   0.0  40.0  1.0    1.0   >50K.
2003  21.0  1.0  16.0  5.0  7315.0   1.0   2.0  40.0  2.0    2.0  <=50K.
2004  28.0  1.0  21.0  3.0  3255.0   1.0   0.0  40.0  2.0    1.0  <=50K.
      AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P   PINCP
2000  35.0  5.0  19.0  1.0  9310.0  51.0   0.0  40.0  1.0    1.0   >50K.
2001  21.0  5.0  16.0  5.0  9825.0  31.0  17.0  60.0  1.0    1.0  <=50K.
2002  44.0  1.0  18.0  1.0  9130.0  53.0   0.0  50.0  1.0    1.0   >50K.
2003  42.0  1.0  17.0  3.0  6720.0  48.0  17.0  84.0  1.0    1.0   >50K.
2004  54.0  4.0  19.0  1.0  6200.0   5.0   1.0  45.0  1.0    1.0   >50K.
      AGEP  COW  SCHL  MAR    OCCP  POBP  RELP  WKHP  SEX  RAC1P   PINCP
2000  57.0  1.0  19.0  3.0   136.0  34.0   0.0  60.

In [28]:
state_sex_race_df = pd.DataFrame(state_sex_race_counts)

state_sex_race_df.to_csv('50_clients_data/state_sex_race_counts.csv', index=False)

state_sex_race_df

,State,Male_Count,Female_Count,White_Count,Black_Count
0,AL,1038,962,1588,412
1,AK,1129,871,1932,68
2,AZ,1052,948,1912,88
3,AR,1026,974,1807,193
4,CA,1070,930,1856,144
5,CO,1064,936,1935,65
6,CT,1013,987,1819,181
7,DE,1004,996,1674,326
8,FL,1025,975,1755,245
9,GA,1037,963,1473,527


In [9]:
AL_df = pd.read_csv('50_clients_data/raw_data/AL.data', header=None)
AL_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,51.0,4.0,21.0,1.0,2002.0,1.0,1.0,40.0,2.0,1.0,<=50K
1,28.0,1.0,20.0,1.0,4110.0,1.0,0.0,14.0,2.0,1.0,<=50K
2,50.0,1.0,16.0,1.0,3300.0,1.0,0.0,40.0,1.0,1.0,>50K
3,58.0,4.0,24.0,5.0,2205.0,138.0,0.0,50.0,2.0,1.0,>50K
4,45.0,1.0,19.0,5.0,4760.0,1.0,0.0,40.0,2.0,2.0,<=50K


In [12]:
AL_df = pd.read_csv('50_clients_data/raw_data/CA.data', header=None)
AL_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,36.0,1.0,16.0,1.0,5330.0,6.0,1.0,40.0,2.0,1.0,>50K
1,41.0,1.0,19.0,1.0,6200.0,303.0,1.0,40.0,1.0,1.0,>50K
2,71.0,1.0,19.0,1.0,600.0,25.0,0.0,12.0,1.0,1.0,>50K
3,40.0,2.0,21.0,1.0,440.0,41.0,0.0,40.0,2.0,1.0,>50K
4,50.0,1.0,12.0,1.0,8320.0,303.0,6.0,40.0,2.0,1.0,<=50K


In [15]:
AL_df[8].value_counts()

8
1.0    1070
2.0     930
Name: count, dtype: int64

In [16]:
AL_df[9].value_counts()

9
1.0    1856
2.0     144
Name: count, dtype: int64

## Torch2 -- Group and Subgroup

In [33]:
# for state_code, df in dfs_loaded.items():
#     df_temp=dfs_loaded[state_code].reset_index(drop=True)
#     train_data, test_data = train_test_split(df_temp, test_size=0.2, random_state=42)

In [34]:
with open('dfs.pickle', 'rb') as f:
    dfs_loaded = pickle.load(f)

merge_df=dfs_loaded["NY"].reset_index(drop=True)

In [35]:
df_temp.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,35.0,1.0,16.0,3.0,6800.0,16.0,0.0,45.0,1.0,1.0,>50K
1,31.0,1.0,22.0,5.0,5600.0,117.0,0.0,36.0,2.0,1.0,<=50K
2,65.0,4.0,24.0,3.0,360.0,19.0,0.0,40.0,1.0,1.0,>50K
3,32.0,1.0,20.0,1.0,350.0,56.0,0.0,50.0,2.0,1.0,>50K
4,48.0,4.0,16.0,1.0,110.0,56.0,1.0,40.0,1.0,1.0,>50K


In [21]:
merge_df=merge_dfs["NY"].copy()

In [22]:
col_names=['AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX', 'RAC1P', 'PINCP']

In [23]:
state_code="NY"
temp_df_train=pd.read_csv(f'50_clients_data/raw_data/{state_code}.data', header=None)
temp_df_test=pd.read_csv(f'50_clients_data/raw_data/{state_code}.test', header=None)

temp_df_train.columns = col_names
temp_df_test.columns = col_names
temp_df_train.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,44.0,1.0,11.0,3.0,4110.0,36.0,2.0,45.0,2.0,1.0,<=50K
1,22.0,1.0,16.0,5.0,4720.0,36.0,2.0,15.0,1.0,2.0,<=50K
2,54.0,1.0,23.0,3.0,3010.0,36.0,0.0,50.0,1.0,1.0,>50K
3,46.0,1.0,22.0,1.0,20.0,138.0,1.0,65.0,2.0,1.0,>50K
4,64.0,6.0,20.0,3.0,4230.0,36.0,11.0,25.0,2.0,1.0,<=50K


In [24]:
len(temp_df_train)

9600

In [25]:
temp_df_test.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,19.0,1.0,15.0,5.0,9620.0,210.0,3.0,15.0,1.0,6.0,<=50K.
1,54.0,1.0,16.0,1.0,5000.0,36.0,1.0,40.0,2.0,1.0,>50K.
2,34.0,1.0,21.0,1.0,9142.0,207.0,10.0,48.0,1.0,6.0,<=50K.
3,56.0,2.0,22.0,1.0,2310.0,36.0,0.0,35.0,2.0,1.0,>50K.
4,55.0,1.0,22.0,5.0,110.0,414.0,0.0,40.0,1.0,1.0,<=50K.


In [26]:
merge_df_train = temp_df_train
merge_df_test = temp_df_test
len(merge_df_train),len(merge_df_test)

(9600, 2400)

In [27]:
merge_df_train.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,44.0,1.0,11.0,3.0,4110.0,36.0,2.0,45.0,2.0,1.0,<=50K
1,22.0,1.0,16.0,5.0,4720.0,36.0,2.0,15.0,1.0,2.0,<=50K
2,54.0,1.0,23.0,3.0,3010.0,36.0,0.0,50.0,1.0,1.0,>50K
3,46.0,1.0,22.0,1.0,20.0,138.0,1.0,65.0,2.0,1.0,>50K
4,64.0,6.0,20.0,3.0,4230.0,36.0,11.0,25.0,2.0,1.0,<=50K


In [28]:
#White Men
WM = merge_df_train.loc[(merge_df_train['SEX'] == 1) & (merge_df_train['RAC1P'] == 1)]
#Black Men
BM = merge_df_train.loc[(merge_df_train['SEX'] == 1) & (merge_df_train['RAC1P'] == 2)]
#white Women
WW = merge_df_train.loc[(merge_df_train['SEX'] == 2) & (merge_df_train['RAC1P'] == 1)]
#Black Women
BW = merge_df_train.loc[(merge_df_train['SEX'] == 2) & (merge_df_train['RAC1P'] == 2)]

print("Length: ",len(WM),len(BM),len(WW),len(BW))

WW.to_csv(f'50_clients_data/client_subG_splits/WW.data', header=False, index=False)
WM.to_csv(f'50_clients_data/client_subG_splits/WM.data', header=False, index=False)
BW.to_csv(f'50_clients_data/client_subG_splits/BW.data', header=False, index=False)
BM.to_csv(f'50_clients_data/client_subG_splits/BM.data', header=False, index=False)

Length:  3537 449 3353 593


In [29]:
WM.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
2,54.0,1.0,23.0,3.0,3010.0,36.0,0.0,50.0,1.0,1.0,>50K
6,41.0,3.0,22.0,1.0,230.0,36.0,0.0,50.0,1.0,1.0,>50K
7,27.0,1.0,21.0,5.0,2640.0,42.0,12.0,45.0,1.0,1.0,>50K
10,52.0,1.0,17.0,1.0,310.0,109.0,1.0,98.0,1.0,1.0,<=50K
11,55.0,1.0,21.0,1.0,5860.0,36.0,0.0,35.0,1.0,1.0,<=50K


In [38]:
# # # filter the dataframe for each combination

# #White Men
# WM_test = merge_df_test.loc[(merge_df_test['SEX'] == 1) & (merge_df_test['RAC1P'] == 1)]
# #Black Men
# BM_test = merge_df_test.loc[(merge_df_test['SEX'] == 1) & (merge_df_test['RAC1P'] == 2)]
# #white Women
# WW_test = merge_df_test.loc[(merge_df_test['SEX'] == 2) & (merge_df_test['RAC1P'] == 1)]
# #Black Women
# BW_test = merge_df_test.loc[(merge_df_test['SEX'] == 2) & (merge_df_test['RAC1P'] == 2)]

# print("Length: ",len(WM_test),len(BM_test),len(WW_test),len(BW_test))

# WW_test.to_csv(f'50_clients_data/client_subG_splits/WW.test', header=False, index=False)
# WM_test.to_csv(f'50_clients_data/client_subG_splits/WM.test', header=False, index=False)
# BW_test.to_csv(f'50_clients_data/client_subG_splits/BW.test', header=False, index=False)
# BM_test.to_csv(f'50_clients_data/client_subG_splits/BM.test', header=False, index=False)

# BM_test.head()

In [39]:
# WM_test['PINCP'] = WM_test['PINCP'].astype(str) + '.'


In [40]:

# # ===== Training =============

# WW_cut= 111
# WM_cut= 111
# BW_cut=111
# BM_cut=111

# # ===== Testing =============


# WW.loc[WW_cut:, 'PINCP'] = WW.loc[WW_cut:, 'PINCP'].astype(str) + '.'
# WW[WW_cut:].to_csv(f'50_clients_data/client_subG_splits/WW.test', header=False, index=False)

# WM.loc[WM_cut:, 'PINCP'] = WM.loc[WM_cut:, 'PINCP'].astype(str) + '.'
# WM[WM_cut:].to_csv(f'50_clients_data/client_subG_splits/WM.test', header=False, index=False)

# BW.loc[BW_cut:, 'PINCP'] = BW.loc[BW_cut:, 'PINCP'].astype(str) + '.'
# BW[BW_cut:].to_csv(f'50_clients_data/client_subG_splits/BW.test', header=False, index=False)

# BM.loc[BM_cut:, 'PINCP'] = BM.loc[BM_cut:, 'PINCP'].astype(str) + '.'
# BM[BM_cut:].to_csv(f'50_clients_data/client_subG_splits/BM.test', header=False, index=False)


# WW[WW_cut:].head()


#-------------- Group data with NY-------------
### subgroup data with NY

In [12]:
merge_df=merge_dfs["NY"].copy()
len(merge_df)

103021

In [13]:
men = merge_df.loc[(merge_df['SEX'] == 1)]
women = merge_df.loc[(merge_df['SEX'] == 2)]
white = merge_df.loc[(merge_df['RAC1P'] == 1)]
black = merge_df.loc[(merge_df['RAC1P'] == 2)]

print("Length: ",len(men),len(women),len(white),len(black))

# =====Training=============
men[:40000].to_csv(f'50_clients_data/client_subG_splits/men.data', header=False, index=False)
women[:40000].to_csv(f'50_clients_data/client_subG_splits/women.data', header=False, index=False)

white[:40000].to_csv(f'50_clients_data/client_subG_splits/white.data', header=False, index=False)
black[:10000].to_csv(f'50_clients_data/client_subG_splits/black.data', header=False, index=False)


# =====Testing=============


men.loc[40000:, 'PINCP'] = men.loc[40000:, 'PINCP'].astype(str) + '.'
men[40000:].to_csv(f'50_clients_data/client_subG_splits/men.test', header=False, index=False)

women.loc[40000:, 'PINCP'] = women.loc[40000:, 'PINCP'].astype(str) + '.'
women[40000:].to_csv(f'50_clients_data/client_subG_splits/women.test', header=False, index=False)

white.loc[40000:, 'PINCP'] = white.loc[40000:, 'PINCP'].astype(str) + '.'
white[40000:].to_csv(f'50_clients_data/client_subG_splits/white.test', header=False, index=False)

black.loc[10000:, 'PINCP'] = black.loc[10000:, 'PINCP'].astype(str) + '.'
black[10000:].to_csv(f'50_clients_data/client_subG_splits/black.test', header=False, index=False)

(52178, 50843, 73665, 11647)

# Take random 100 data points from Clients

In [13]:
import os

In [14]:
# Training Ground Truth

In [4]:
folder_path = '50_clients_data/raw_data/'
for file_name in os.listdir(folder_path):
    if file_name.endswith('.data'):
        df = pd.read_csv(os.path.join(folder_path, file_name), header=None)
        temp_df=df.sample(n=100)
        base_name = os.path.splitext(file_name)[0]
        temp_df.to_csv(f'50_clients_data/client_raw_data_100_sample/{base_name}_100.data', header=False, index=False)


In [16]:
# Testing

In [5]:
folder_path = '50_clients_data/raw_data/'
for file_name in os.listdir(folder_path):
    if file_name.endswith('.test'):
        df = pd.read_csv(os.path.join(folder_path, file_name), header=None)
        temp_df=df.sample(n=100)
        base_name = os.path.splitext(file_name)[0]
        temp_df.to_csv(f'50_clients_data/client_raw_data_100_test/{base_name}_100.test', header=False, index=False)

In [19]:
# for state_code, df in dfs_loaded.items():
#     df_temp=dfs_loaded[state_code].reset_index(drop=True)
#     train_data, test_data = train_test_split(df_temp, test_size=0.2, random_state=42)
    
#     train_data.to_csv(f'50_clients_data/client_raw_data_100_sample/{state_code}_100.data', header=False, index=False)

# sample_100_AK.to_csv(f'50_clients_data/raw_data/AK_100.data', header=False, index=False)

In [21]:
# for state_code, (features, label) in dfs.items():
#     # take 30%
#     num_rows_to_keep = int(len(features) * 0.3) 
#     random_indices = np.random.choice(len(features), num_rows_to_keep, replace=False)
#     reduced_features = features.iloc[random_indices]
#     reduced_label = label.iloc[random_indices]
#     dfs[state_code] = (reduced_features, reduced_label)

# for state_code, (reduced_features, reduced_label) in dfs.items():
#     print(f"State: {state_code}, Reduced Features Length: {len(reduced_features)}, Reduced Label Length: {len(reduced_label)}")

In [16]:
import pickle

# Save the dictionary to a file
with open('dfs.pickle', 'wb') as f:
    pickle.dump(merge_dfs, f)

In [17]:
with open('dfs.pickle', 'rb') as f:
    dfs_loaded = pickle.load(f)

In [18]:
for state_code, df in dfs_loaded.items():
    print(f"State: {state_code}, df Length: {len(df)}")

State: AL, df Length: 22268


In [20]:
dfs_loaded["AL"].reset_index(drop=True)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,18.0,1.0,18.0,5.0,4720.0,13.0,17.0,21.0,2.0,2.0,<=50K
1,53.0,5.0,17.0,5.0,3605.0,18.0,16.0,40.0,1.0,1.0,<=50K
2,41.0,1.0,16.0,5.0,7330.0,1.0,17.0,40.0,1.0,1.0,<=50K
3,18.0,6.0,18.0,5.0,2722.0,1.0,17.0,2.0,2.0,1.0,<=50K
4,21.0,5.0,19.0,5.0,3870.0,12.0,17.0,50.0,1.0,1.0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
22263,20.0,6.0,19.0,5.0,4251.0,1.0,4.0,25.0,1.0,1.0,<=50K
22264,63.0,1.0,16.0,1.0,440.0,1.0,0.0,48.0,1.0,1.0,>50K
22265,65.0,2.0,21.0,5.0,420.0,1.0,2.0,40.0,2.0,1.0,>50K
22266,37.0,1.0,16.0,4.0,340.0,6.0,0.0,50.0,2.0,1.0,<=50K
